In [27]:
#############
## imports ##
#############

# libraries 
import geopandas as gpd
import numpy as np 
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask
import os
import json
from shapely.geometry import box, Polygon
import logging

##########
## code ##
##########

def tif_from_ruta(ruta_geometry):
    minx_ruta = ruta_geometry.bounds[0]
    miny_ruta = ruta_geometry.bounds[1]

    miny = str(miny_ruta)[0:3]
    minx = str(minx_ruta)[0:2]
    km_siffran_y = 0 if int(str(miny_ruta)[3]) < 5 else 5
    km_siffran_x = 0 if int(str(minx_ruta)[2]) < 5 else 5
    year = 2018

    filename = f"{miny}_{minx}_{km_siffran_y}{km_siffran_x}_{year}.tif"
    return filename


def filter_imgs(all_rutor_path, dir_files):
    all_rutor = gpd.read_file(all_rutor_path)
    all_rutor['in_tif'] = all_rutor['geometry'].map(tif_from_ruta)
    uniques = all_rutor.in_tif.unique()

    # dir_files = os.listdir(original_tif_dir)
    only_tifs = [filename for filename in dir_files if filename[-4:] == ".tif"]

    only_tifs_noyear = [filename[:-8] for filename in only_tifs]
    uniques_noyear = [filename[:-8] for filename in list(uniques)]

    # check that all uniques are in only tifs
    if not (set(list(uniques_noyear)).issubset(set(only_tifs_noyear))):
        # logger.WARN(f"at least one tif name generated from all_rutor was not found in the directory: {original_tif_dir}")
        print(f"at least one tif name generated from all_rutor was not found in the directory")
        items_not_in_dir = [item for item in list(uniques) if item not in only_tifs]
        print(f"items not in directory are: \n {items_not_in_dir}")

    intersection = list(set(uniques_noyear) & set(only_tifs_noyear))

    tifs_to_use = [filename for filename in only_tifs if filename[:-8] in intersection]

    return tifs_to_use

In [22]:
with open('/Users/nadja/Documents/UU/Thesis/filelist.txt', 'r') as file:
    lines = file.readlines()

items = []
for line in lines:
    item = line.strip()
    item = f"{item}f"
    
    items.append(item)

In [28]:
shape_file = "/Users/nadja/Documents/UU/Thesis/data/Vector/palsaShape/RUTNAT_100x100M_PALS_OR_PALSVATT_ALL.shp"
filtered_imgs = filter_imgs(shape_file, items)

In [30]:
len(filtered_imgs)

782

In [33]:
imgs_to_use = pd.DataFrame(filtered_imgs)

In [36]:
imgs_to_use.to_csv("filenames_to_use.csv", header = False, index = False)

In [35]:
!pwd

/Users/nadja/Documents/UU/Thesis/palsa_seg/data_prep


I have a list of filenames that i want to compare to another list of filenames. Specifically, I will need to use the files which are present in both lists. Both lists have filenames that are named according to the following format: "{filename}_{year}.tif". The filenames are unique, but some of the years are different in list1 versus list2. This is an error, such that i want the intersection of the 2 lists, ignoring the {year} component of the name. Is there any way to achieve this? 

